# Description
This file is used to convert abbriviation on name back into dictionary words. There are two approaches that we are going to try: the first approach is to use a dictionary that maps abbriviations to dictionary words. The second approach is pass it into LLMs and see how would the LLM guess. 

## Read Files
First we will read the result from previous work, in which we get python files from internet, parse them to get all the variable and function names (and their scope, might be useful later), and then we parse the variable and function names into terms (in a primitive way). 

In [1]:
import pandas as pd
import sqlite3
name_table = "NameTable"
conn = sqlite3.connect('data.db')
query = f"SELECT * FROM {name_table}"
df = pd.read_sql_query(query, conn)
import json
df['terms'] = df.terms.apply(json.loads)
df

,id,name,nameType,nameScope,projectSize,authorName,authorProficiency,authorLocation,terms,namingConvention
0,0,_raise_err,function,GlobalScope,72400,programthink,<50,China,"[raise, err]",Snake
1,1,_load_yaml,function,GlobalScope,72400,programthink,<50,China,"[load, yaml]",Snake
2,2,Node,class,GlobalScope,72400,programthink,<50,China,[Node],Pascal
3,3,Relation,class,GlobalScope,72400,programthink,<50,China,[Relation],Pascal
4,4,Family,class,GlobalScope,72400,programthink,<50,China,[Family],Pascal
...,...,...,...,...,...,...,...,...,...,...
8234210,8234210,__long__,variable,FunctionScope,10460,juvers,>100,USA,[long],Snake
8234211,8234211,__getslice__,variable,FunctionScope,10460,juvers,>100,USA,[getslice],Snake
8234212,8234212,__setslice__,variable,FunctionScope,10460,juvers,>100,USA,[setslice],Snake
8234213,8234213,__delslice__,variable,FunctionScope,10460,juvers,>100,USA,[delslice],Snake


## Get terms and frequency

In [2]:
# get all the terms in Chinese dataframe
flattened_words_chinese = df[df['authorLocation'] == 'China']['terms'].explode()
# Count the occurrences of each unique terms
word_counts_chinese = flattened_words_chinese.value_counts()
df_chinese_word_freq = pd.DataFrame(word_counts_chinese.reset_index())

# do the same for Americans
flattened_words_english = df[df['authorLocation'] == 'USA']['terms'].explode()
# Count the occurrences of each word
word_counts_english = flattened_words_english.value_counts()
df_english_word_freq = pd.DataFrame(word_counts_english.reset_index())

# do the total words
flattened_words = df['terms'].explode()
# Count the occurrences of each word
word_counts = flattened_words.value_counts()
df_word_freq = pd.DataFrame(word_counts.reset_index())

In [3]:
df_word_freq

,terms,count
0,test,366742
1,init,173722
2,get,171985
3,name,113341
4,ID,97638
...,...,...
138447,bistochastize,1
138448,WKTBASES,1
138449,Toward,1
138450,MICROS,1


## Create a dictionary
Let's try to use the nltk dictionary to determine if a word is a real english word

In [4]:
from nltk.corpus import words

# english_dictionary = set(words.words())

# I will use a better dictionary: ENABLE (Enhanced North American Benchmark Lexicon)
with open('SavedFiles/atebits.txt', 'r') as file:
    words = file.read().splitlines()
english_dictionary =  set(words)



In [5]:
len(english_dictionary)

274926

Let's see how many real words are in the total terms

In [6]:
def lookup_terms(term):
    return term.lower() in english_dictionary

def percentage_of_real_word_unique(df_word_frequency, lookup_func=lookup_terms):
    # assuming that the df_word_frequency have "terms" column and "count" column
    df_word_frequency['real_word'] = df_word_frequency['terms'].apply(lookup_func)
    return df_word_frequency['real_word'].mean() * 100

def percentage_of_real_word(df_word_frequency, lookup_func=lookup_terms):
    # assuming that the df_word_frequency have "terms" column and "count" column
    df_word_frequency['real_word'] = df_word_frequency['terms'].apply(lookup_func)
    return (df_word_frequency[df_word_frequency['real_word'] == 1]['count'].sum() / df_word_frequency['count'].sum())*100

def number_of_real_word_unique(df_word_frequency, lookup_func=lookup_terms):
    # assuming that the df_word_frequency have "terms" column
    df_word_frequency['real_word'] = df_word_frequency['terms'].apply(lookup_func)
    return df_word_frequency['real_word'].sum()

def number_of_real_word(df_word_frequency, lookup_func=lookup_terms):
    # assuming that the df_word_frequency have "terms" column
    df_word_frequency['real_word'] = df_word_frequency['terms'].apply(lookup_func)
    return (df_word_frequency[df_word_frequency['real_word'] == 1]['count'].sum()) *100

In [7]:
for df_word_frequency, df_name in [(df_chinese_word_freq, 'Chinese'), (df_english_word_freq, 'English'), (df_word_freq, 'total')]:
    print(f"For {df_name}, before tokenization: ")
    print("Number of real words:")
    print(number_of_real_word(df_word_frequency))
    print("Number of unique real words: ")
    print(number_of_real_word_unique(df_word_frequency))
    print("Percentage of real words in corpus: ")
    print(percentage_of_real_word(df_word_frequency))
    print("Percentage of unique real words in all unique real words: ")
    print(percentage_of_real_word_unique(df_word_frequency))
    print("======================================")

For Chinese, before tokenization: 
Number of real words:
535400300
Number of unique real words: 
25923
Percentage of real words in corpus: 
72.6715592126284
Percentage of unique real words in all unique real words: 
32.33866842978506
For English, before tokenization: 
Number of real words:
536840000
Number of unique real words: 
29707
Percentage of real words in corpus: 
72.67408804064247
Percentage of unique real words in all unique real words: 
28.432376558866036
For total, before tokenization: 
Number of real words:
1072240300
Number of unique real words: 
35476
Percentage of real words in corpus: 
72.67282530236845
Percentage of unique real words in all unique real words: 
25.623320717649438


## Tokenization
I noticed that the dictionary determines that some conjugation of a word , such as "names" (plural), and "expected" (past tense) is not a word. Thanks to the almighty ChatGPT, we can use lemmatization of check if a word is a real word. 

In [8]:
import nltk
from nltk.stem import WordNetLemmatizer

# Download WordNet if not already downloaded
nltk.download('wordnet')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Check if a word is in the NLTK words corpus after lemmatization
def is_english_word(word):
    base_word = lemmatizer.lemmatize(word.lower())
    return base_word in english_dictionary

[nltk_data] Downloading package wordnet to /Users/samyiin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Why don't we change the original words in the df? Because we might need to analyze the use of plural and tenses (or other conjugations) later. 

In [9]:
for df_word_frequency, df_name in [(df_chinese_word_freq, 'Chinese'), (df_english_word_freq, 'English'), (df_word_freq, 'total')]:
    print(f"For {df_name}, after tokenization: ")
    print("Number of real words:")
    print(number_of_real_word(df_word_frequency, lookup_func=is_english_word))
    print("Number of unique real words: ")
    print(number_of_real_word_unique(df_word_frequency,lookup_func=is_english_word))
    print("Percentage of real words in corpus: ")
    print(percentage_of_real_word(df_word_frequency, lookup_func=is_english_word))
    print("Percentage of unique real words in all unique real words: ")
    print(percentage_of_real_word_unique(df_word_frequency, lookup_func=is_english_word))
    print("======================================")

For Chinese, after tokenization: 
Number of real words:
534645700
Number of unique real words: 
25971
Percentage of real words in corpus: 
72.56913499175693
Percentage of unique real words in all unique real words: 
32.398547922306356
For English, after tokenization: 
Number of real words:
535744700
Number of unique real words: 
29734
Percentage of real words in corpus: 
72.52581308231055
Percentage of unique real words in all unique real words: 
28.45821808332456
For total, after tokenization: 
Number of real words:
1070390400
Number of unique real words: 
35549
Percentage of real words in corpus: 
72.54744532968243
Percentage of unique real words in all unique real words: 
25.67604657209719


Simpson Paradox?

## Single letter
Try to get rid of singgle letters and see how it goes

In [10]:
for df_word_frequency, df_name in [(df_chinese_word_freq, 'Chinese'), (df_english_word_freq, 'English'), (df_word_freq, 'total')]:
    df_word_frequency = df_word_frequency[df_word_frequency['terms'].apply(lambda x: len(x) != 1)].copy()
    print(f"For {df_name}, after tokenization: ")
    print("Number of real words:")
    print(number_of_real_word(df_word_frequency, lookup_func=is_english_word))
    print("Number of unique real words: ")
    print(number_of_real_word_unique(df_word_frequency,lookup_func=is_english_word))
    print("Percentage of real words in corpus: ")
    print(percentage_of_real_word(df_word_frequency, lookup_func=is_english_word))
    print("Percentage of unique real words in all unique real words: ")
    print(percentage_of_real_word_unique(df_word_frequency, lookup_func=is_english_word))
    print("======================================")

For Chinese, after tokenization: 
Number of real words:
534645700
Number of unique real words: 
25971
Percentage of real words in corpus: 
77.25939858424789
Percentage of unique real words in all unique real words: 
32.41957832453282
For English, after tokenization: 
Number of real words:
535744700
Number of unique real words: 
29734
Percentage of real words in corpus: 
76.38633116707206
Percentage of unique real words in all unique real words: 
28.47238846702608
For total, after tokenization: 
Number of real words:
1070390400
Number of unique real words: 
35549
Percentage of real words in corpus: 
76.81993615792668
Percentage of unique real words in all unique real words: 
25.685693641618496


improved by just a little

## translate the word
Since there is only 138k unique terms, around 100k unknown words, we can try to use LLM to predict the word. 
Each term might have more than one occurances in the corpus, for example, the term "init" occurs in 173722 names. What we will do here is we will just select one occurance, and use it as a context for the LLM to guess the original word of the name. 

In [11]:
# we need to save this result, because it takes about 2 mins to run: 
import joblib
memory = joblib.Memory(location='CacheFunctionCalls', verbose=1)
# but probelem is, everytime we restart kernal it reruns.... So we rewrite the decorator. 
def cache(memory, module, **mem_kwargs):
    def cache_(f):
        f.__module__ = module
        f.__qualname__ = f.__name__
        return memory.cache(f, **mem_kwargs)
    return cache_

# cache the function call
@cache(memory, "04_abbriviations")
def get_dic_terms_to_index():
    # we will use df from outer scope
    # create a mapping between a term to it's corresponding rows in df
    dic_term_to_rows = df.explode("terms").groupby('terms')['terms'].apply(lambda x: x.index.tolist()).to_dict()
    return dic_term_to_rows

Now we will take a row (randomly? For now we take the first occurance), and make that row the context of the term

In [12]:
dic_term_to_rows = get_dic_terms_to_index()

def get_context(term):
    # we will use df from outer scope
    # there should not be any key_error because we create the dic from the same df
    row_number = dic_term_to_rows[term][0]
    return df.iloc[row_number]['name']

# for each word in df_word_freq, we will get the first occurance of it as the context
df_word_freq['context'] = df_word_freq['terms'].apply(get_context)

## Query LLM

In [13]:
# give prompts
first_prompt = """I have a list of (term, variable_name) touples, the term appears in the variable_name, variable_name is extracted from python programs. 
The term could be an abbreviation, acronym of words, or concatenation of words, I need you to guess what the term represents.
If you don't know what the term represents, then give -1 as your answer. 
Your output is linked to a computer program, so your output should be in format of a list of strings. Each string corresponds to a touple.
Do you understand?"""
first_respond = "Yes, I fully understand and I will only respond with a string literal of python list. Please provide the list of (variable_names, term) tuples."

We will use the idea of In context learning/ n-shots learning: In-context Learning (ICL; Brown et al., 2020), n-shots learning

In [14]:
example1 = '[(init,init),(x,x),(y,y),(a,a),(s,token_pretrained_s2G),(msg,http_error_msg),(d,d),(X,diffX),(n,n),(url,txt2img_url)]'
answer1 = "['initialization', '-1', '-1', '-1', '-1', 'message', '-1', 'X', '-1', 'Uniform Resource Locator']"

example2 = '[(c,c),(str,__str__),(b,b),(args,arrow_args),(f,delta_f),(config,config),(p,p),(num,num_classes),(dir,data_dir),(t,calc_gradient_t)]'
answer2 = "['-1', 'string', '-1', 'arguments', '-1', 'configuration', '-1', 'number', 'directory', '-1']"

example3 = '[(r,r),(m,m),(repr,__repr__),(i,start_i),(D,train_pretrained_s2D),(v,v_fps),(func,func),(df,df),(iter,__iter__),(params,extract_params)]'
answer3 = "['-1', '-1', 'representation', 'index', 'dimention', '-1', 'function', 'dataframe', 'iterator', 'parameters']"

example4 = '[(webinf,webinf_path),(pxtc,all_data_pca_pxtc),(datatore,expect_datatore_lookup),(preimported,preimported_locals),(camerasfile,camerasfile),(camdata,camdata),(imagesfile,imagesfile),(bistochastize,bistochastize),(WKTBASES,WKTBASES),(itrin,idx_itrin)]'

In [15]:
def read_access_token(file_path="OpenAIAPIKey"):
    """
    Put your access token in an empty file named: OpenAIAPIKey under the same directory of this file (Assume running
    this script from the same directory where it located
    :param file_path:
    :return:
    """
    with open(file_path, 'r') as file:
        access_token = file.read().strip()
    return access_token

In [16]:
import openai
import ast

# get API key from a file (We could have put them in a env variable, I just don't want to bother
API_KEY = read_access_token()

def try_guess(str_list_term_name_tuples):
    client = openai.OpenAI(api_key=API_KEY)
    response = client.chat.completions.create(
        # model = "gpt-4o",
        model = "gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": first_prompt},
            {"role": "assistant", "content": first_respond},
            {"role": "user", "content": example1},
            {"role": "assistant", "content": answer1},
            {"role": "user", "content": example2},
            {"role": "assistant", "content": answer2},
            {"role": "user", "content": example3},
            {"role": "assistant", "content": answer3},
            {"role": "user", "content": str_list_term_name_tuples}])
    return response
    
def accept_response(response):
    res_stop = True
    res_correct = True
    # first check if the response is correct
    if not response.choices[0].finish_reason == "stop":
        res_stop = False
    # now let's check if the response is correct
    gpt_res = response.choices[0].message.content
    try:
        actual_list = ast.literal_eval(gpt_res)
        # now check if the response have the same size as the input 
        # (assume there is 10: the test will fail if we are running that last iter)
        if len(actual_list) != 10:
            res_correct = False
    except:
        res_correct = False
    return res_stop and res_correct
    
# we will cache the results
@cache(memory, "04_abbriviations")
def guess_abbreviation_by_bulk(str_list_term_name_tuples):
    """
    : param: str_list_term_name_tuples: list of tuples as in examples above
    """
    response = try_guess(str_list_term_name_tuples)
    save_money_counter = 1
    while not accept_response(response):
        print("response not accepted")
        response = try_guess(str_list_term_name_tuples)
        save_money_counter += 1
        if save_money_counter > 10:
            # so that the result is not cached
            raise ValueError
    # if it gets to this point then it's correct, ast.literal_eval will succeed
    return ast.literal_eval(response.choices[0].message.content)


In [17]:
guess_abbreviation_by_bulk(example4)

['web information',
 '-1',
 'data store',
 'pre-imported',
 'cameras file',
 'camera data',
 'images file',
 'bistochastize',
 'WKT BASES',
 'iteration index']

In [18]:
df_abbrev_words = df_word_freq[df_word_freq['real_word']==False][['terms', 'context']]

len(df_abbrev_words)

102903

In [19]:
from concurrent.futures import ThreadPoolExecutor, as_completed

df_abbrev_words = df_word_freq[df_word_freq['real_word']==False][['terms', 'context']]

def pad_term_name_tuples(name_term_tuples):
    '''
    First of all, the name_term_tuples is actually term_name_tuples, terms is the first column, 
    Since the guess_abbreviation_by_bulk is taking a string of 10, when the input is not exactly 10, we will pad it. 
    '''
    current_length = len(name_term_tuples)
    target_length = 10
    if current_length < target_length:
        # Calculate the number of tuples to add
        num_tuples_to_add = target_length - current_length
        # Extend the list with (0, 0) tuples
        name_term_tuples.extend(['(0, 0)'] * num_tuples_to_add)
    return name_term_tuples


def init_abbrev_map(df_abbrev_words):
    for i in range((len(df_abbrev_words) // 10)+1):
        start = i * 10
        end = start + 10
        # get the next bulk of abbreviated words
        name_term_tuples = ['('+','.join([row[0],row[1]])+')' for row in df_abbrev_words[start:end].to_numpy()]
        name_term_tuples = pad_term_name_tuples(name_term_tuples)
        str_list_term_name_tuples = '[' + ','.join(name_term_tuples) + ']'
        # get respond from LLM: we will cache the result, so there is no need to record the output
        guess_abbreviation_by_bulk(str_list_term_name_tuples)

import time
def init_abbrev_map_robust(df_abbrev_words):
    '''This function is robust to internet fail and ValueError from ChatGPT'''
    weird_inputs = []
    while True:
            try:
                init_abbrev_map(df_abbrev_words[:])
                break  # Exit the loop if function executes successfully
            except ValueError as e:
                weird_inputs.append(str_list_term_name_tuples)
                print("found weird input")
            except Exception as e:
                print(f"Exception occurred: {e}")
                print("Retrying function...")
                time.sleep(50)  # Wait for a while before retrying
            
    return weird_inputs



In [20]:
def clear_err_outputs():
    """
    In  my previous run I forgot to check if the result from chat gpt is of the right length, 
    So I am fixing that error here, now I have fixed the error, this function should only be ran once
    """
    for i in range(len(df_abbrev_words) // 10):
        start = i * 10
        end = start + 10
        # get the next bulk of abbreviated words
        name_term_tuples = ['('+','.join([row[0],row[1]])+')' for row in df_abbrev_words[start:end].to_numpy()]
        str_list_term_name_tuples = '[' + ','.join(name_term_tuples) + ']'
        # get respond from LLM: we will cache the result, so there is no need to record the output
        list_abbrev_meaning = guess_abbreviation_by_bulk(str_list_term_name_tuples)
        if len(list_abbrev_meaning) != 10:
            # clear the caches
            result = guess_abbreviation_by_bulk.call_and_shelve(str_list_term_name_tuples)
            result.clear()
        
# clear_err_outputs()

Retrospect here: taking a string representation of a list as input is really a bad design. If I take input in as a set, then if what's passed in is a subset, it would not rerun everything. 

### Abbrive map initialzation (0 iteration)

In [21]:
import ast

df_abbrev_words = df_word_freq[df_word_freq['real_word']==False][['terms', 'context']]
    
def get_abbrev_map(df_abbrev_words, weird_inputs):
    list_all_terms = []
    list_all_names = []
    list_all_abbrev_meaning = []
    for i in range((len(df_abbrev_words) // 10)+1):
        start = i * 10
        end = start + 10
        # get the next bulk of abbreviated words
        name_term_tuples = ['('+','.join([row[0],row[1]])+')' for row in df_abbrev_words[start:end].to_numpy()]
        name_term_tuples = pad_term_name_tuples(name_term_tuples)
        str_list_term_name_tuples = '[' + ','.join(name_term_tuples) + ']'
        # during init there are weird inputs that ChatGPT just cannot get it right, so we will not rerun that. 
        if str_list_term_name_tuples in weird_inputs:
            list_abbrev_meaning = ['-1'] * 10
        else:
            list_abbrev_meaning = guess_abbreviation_by_bulk(str_list_term_name_tuples)
        list_all_abbrev_meaning.extend(list_abbrev_meaning)        
        # create the list of names
        list_terms = [row[0] for row in df_abbrev_words[start:end].to_numpy()]
        list_all_terms.extend(list_terms)
        list_names = [row[1] for row in df_abbrev_words[start:end].to_numpy()]
        list_all_names.extend(list_names) 
    df_abbrev_map = pd.DataFrame({
    'term': list_all_terms,
    'name': list_all_names,
    'abbrev_meaning': list_all_abbrev_meaning[: len(df_abbrev_words)]
    })
    return df_abbrev_map

weird_inputs = init_abbrev_map_robust(df_abbrev_words)
df_abbrev_map = get_abbrev_map(df_abbrev_words, weird_inputs)
df_abbrev_map

,term,name,abbrev_meaning
0,init,init,initialization
1,x,x,-1
2,y,y,-1
3,a,a,-1
4,s,token_pretrained_s2G,-1
...,...,...,...
102898,camdata,camdata,-1
102899,imagesfile,imagesfile,images file
102900,bistochastize,bistochastize,-1
102901,WKTBASES,WKTBASES,well-known text bases


Something that is definitely worth noting is that there are certain inputs that is just hard for chatGPT to find answer for. And the weird thing here is that they are usually not the long ones or the complex ones. Characterizing them may leads to so good results?

In [22]:
'''There is something I forgot to check, and now it's too late to change because I have cached all the results, 
So I have to change it here now: the return of chat GPT might be '-1' (str) or -1 (int). '''
def convert_neg_one(value):
    if isinstance(value, str) and value.strip() == '-1':
        return -1
    return value
df_abbrev_map['abbrev_meaning'] = df_abbrev_map['abbrev_meaning'].apply(convert_neg_one)

df_abbrev_map_success = df_abbrev_map[df_abbrev_map['abbrev_meaning']!= -1]
print(df_abbrev_map_success.shape[0], df_abbrev_map.shape[0])

62315 102903


### Abbrieve map iteration
There are more abbreviations that ChatGPT should have recognized, So in order to find as many as possible, I would like to try a different approach. There are two things to balance here: I don't want chatGPT to recognize it should't know, for example, recognize a as apple. On the other hand, I want chatGPT to recognize things it should know, like, webinf is web information.  
I can't think of any good solution at this point, what I can think of, is to run it multiple times one different inputs, iteratively, and hoping that it can recognize more and more...  
Besides, I will change the prompt, reduce the example where there is '-1' in the results, hoping that ChatGPT will not recognize it as a pattern. Because in my examples there are 14/30 '-1', and in the result, there are almost the same percenatge of '-1', so I suspect that.  
2024.06.06 I observe that this is improving the results, but it is too "confidient" in a sense that would predict a as alpha. Let me first use this method, and then determine should I keep the single letter's interpretation: because sometimes it would also predict s to sequence, which is chorrect under context token_pretrained_s2G.  
Now, finally, we ran 40k and we left with 4k unknown. So I guess providing new context deos help. 

In [23]:
def generating_examples(start, df_abbrev_map):
    '''In the first iteration, I have to type out examples manually, but I get some results, 
    so now I will generate examples by what I already have, the manual examples will be overritten at this point.'''
    end = start + 10
    df_abbrev_map_success = df_abbrev_map[df_abbrev_map['abbrev_meaning']!= '-1']
    name_term_tuples = ['('+','.join([row[0],row[1]])+')' for row in df_abbrev_map[start:end].to_numpy()]
    str_list_term_name_tuples = '[' + ','.join(name_term_tuples) + ']'
    results = ["'" + row[2] + "'" for row in df_abbrev_map_success[start:end].to_numpy()]
    str_list_results = '[' + ','.join(results) + ']'
    return str_list_term_name_tuples, str_list_results

# this will overrite the examples above
example1, answer1 = generating_examples(0, df_abbrev_map_success)
example2, answer2 = generating_examples(10, df_abbrev_map_success)
example3, answer3 = generating_examples(20, df_abbrev_map_success)

In [34]:
example2, answer2

('[(i,start_i),(D,train_pretrained_s2D),(func,func),(df,df),(iter,__iter__),(params,extract_params),(obj,nmap_obj),(img,img),(val,val),(idx,idx)]',
 "['index','dimension','function','dataframe','iterator','parameters','object','image','value','index']")

In [24]:
def add_abbrev_meaning(row):
    '''assuming that the row have 'abbrev_meaning' and 'abbrev_meaning_to_add' '''
    if row['abbrev_meaning'] == -1 :
        row['abbrev_meaning'] = row['abbrev_meaning_to_add']
    return row

def iterative_fill_in_abbriv_map(df_abbrev_map, n_iter):
    for i in range(n_iter):
        df_abbrev_map_fail = df_abbrev_map[df_abbrev_map['abbrev_meaning'] == -1]
        weird_inputs = init_abbrev_map_robust(df_abbrev_map_fail)
        df_abbrev_map_mixed = get_abbrev_map(df_abbrev_map_fail, weird_inputs)
        
        # merge the new result with the old result -- df_abbrev_map
        df_abbrev_map = pd.merge(df_abbrev_map, df_abbrev_map_mixed, how='left', on=['term', 'name'], suffixes=('', '_to_add'))
        df_abbrev_map = df_abbrev_map.apply(add_abbrev_meaning, axis=1)
        # convert '-1' to -1, trim the columns
        df_abbrev_map = df_abbrev_map[['term', 'name', 'abbrev_meaning']]
        df_abbrev_map['abbrev_meaning'] = df_abbrev_map['abbrev_meaning'].apply(convert_neg_one)
    return df_abbrev_map

# don't run this function twice, it will continue running

df_abbrev_map = iterative_fill_in_abbriv_map(df_abbrev_map, n_iter=3)

In [35]:
df_abbrev_map

,term,name,abbrev_meaning
0,init,init,initialization
1,x,x,x
2,y,y,y
3,a,a,alpha
4,s,token_pretrained_s2G,token pretrained sequence to graph
...,...,...,...
102898,camdata,camdata,camera data
102899,imagesfile,imagesfile,images file
102900,bistochastize,bistochastize,convert to bistochastic matrix
102901,WKTBASES,WKTBASES,well-known text bases


In [25]:
df_abbrev_map_fail = df_abbrev_map[df_abbrev_map['abbrev_meaning'] == -1]
df_abbrev_map_fail

,term,name,abbrev_meaning
17596,nokey,test_read_nokey,-1
22080,truncexpon,truncexpon_gen,-1
22150,Renderables,Renderables,-1
22223,cfm,cfm,-1
25508,funm,funm,-1
...,...,...,...
101195,univgw,_univgw,-1
101197,argcounts,vtbl_argcounts,-1
101530,MASKEDSTRUCT,_WIN32MASKEDSTRUCT,-1
101532,lvc,lvc,-1


after 0 iter: 40k  
after 1 iter: 4k  
after 2 iter: 800  
after 3 iter: 277  
There are several problems with this approach:
1. predicting "too much": a -> alpha, h -> height
2. There is no mapping for each instance: we selected just one instance for each abbreviation.
3. Didn't recognize: MASKEDSTRUCT = masked sctruct
4. Output error: (s,token_pretrained_s2G) -> "token pretrained sequence to graph"

### Store results to database

In [26]:
df_abbrev_map

,term,name,abbrev_meaning
0,init,init,initialization
1,x,x,x
2,y,y,y
3,a,a,alpha
4,s,token_pretrained_s2G,token pretrained sequence to graph
...,...,...,...
102898,camdata,camdata,camera data
102899,imagesfile,imagesfile,images file
102900,bistochastize,bistochastize,convert to bistochastic matrix
102901,WKTBASES,WKTBASES,well-known text bases


In [27]:
df_abbrev_map = df_abbrev_map.astype(str)
df_abbrev_map

,term,name,abbrev_meaning
0,init,init,initialization
1,x,x,x
2,y,y,y
3,a,a,alpha
4,s,token_pretrained_s2G,token pretrained sequence to graph
...,...,...,...
102898,camdata,camdata,camera data
102899,imagesfile,imagesfile,images file
102900,bistochastize,bistochastize,convert to bistochastic matrix
102901,WKTBASES,WKTBASES,well-known text bases


In [28]:
abbrev_table = "AbbreviationMap"
# sqlite connect is also relative path relative to the folder running the script.
conn = sqlite3.connect('data.db')
# store results to the table
df_abbrev_map.to_sql(abbrev_table, conn, index=False, if_exists='replace')
conn.commit()
conn.close()

In [29]:
# sqlite connect is also relative path relative to the folder running the script.
conn = sqlite3.connect('data.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
table_names = cursor.fetchall()
print([name[0] for name in table_names])

['NameTable', 'AbbreviationMap']


In [30]:
# conn = sqlite3.connect('data.db')
# abbrev_table = "AbbreviationMap"

# cursor = conn.cursor()
# cursor.execute(f"DROP TABLE IF EXISTS {abbrev_table}")

In [31]:
conn = sqlite3.connect('data.db')
query = f"SELECT * FROM {abbrev_table}"
df = pd.read_sql_query(query, conn)

In [32]:
df

,term,name,abbrev_meaning
0,init,init,initialization
1,x,x,x
2,y,y,y
3,a,a,alpha
4,s,token_pretrained_s2G,token pretrained sequence to graph
...,...,...,...
102898,camdata,camdata,camera data
102899,imagesfile,imagesfile,images file
102900,bistochastize,bistochastize,convert to bistochastic matrix
102901,WKTBASES,WKTBASES,well-known text bases


Later might need to mannually go through some of the cases......